# CityBikes

Send a request to CityBikes for the city of your choice. 

In [125]:
import pandas as pd 
from pandas import json_normalize
import json
import requests
import citybikes as cb
from datetime import datetime

In [82]:
## Method 1: JSON

In [83]:
city_bikes = requests.get('http://api.citybik.es/v2/networks').json()

In [84]:
city_bikes['networks'][28]

{'company': ['Bike U Sp. z o.o.'],
 'href': '/v2/networks/bikeu-bra',
 'id': 'bikeu-bra',
 'location': {'city': 'Bydgoszcz',
  'country': 'PL',
  'latitude': 53.12193,
  'longitude': 18.00038},
 'name': 'Bydgoski rower aglomeracyjny'}

In [85]:
city_bikes['networks'][14]['location']['country']

'IT'

In [86]:
def get_city_data(city): 
    city_bike_networks = requests.get("http://api.citybik.es/v2/networks").json()

    list_of_dicts = []
    for city_bike_dict in city_bike_networks['networks']:
        new_city = city_bike_dict['location']['city']
        if new_city.lower() == city.lower(): 
            list_of_dicts.append(city_bike_dict)
    
    return list_of_dicts

In [137]:
city_names = ['Moscow', 'Barcelona', 'London', 'Paris', 'Glasgow', 'Toulouse', 'Bydgoszcz']

for each_city in city_names: 
    print(get_city_data(each_city))
    print("----------------------------/n")

[{'company': ['ЗАО «СитиБайк»'], 'href': '/v2/networks/velobike-moscow', 'id': 'velobike-moscow', 'location': {'city': 'Moscow', 'country': 'RU', 'latitude': 55.75, 'longitude': 37.616667}, 'name': 'Velobike'}]
----------------------------/n
[{'company': ['Barcelona de Serveis Municipals, S.A. (BSM)', 'CESPA', 'PBSC'], 'gbfs_href': 'https://api.bsmsa.eu/ext/api/bsm/gbfs/v2/', 'href': '/v2/networks/bicing', 'id': 'bicing', 'location': {'city': 'Barcelona', 'country': 'ES', 'latitude': 41.3850639, 'longitude': 2.1734035}, 'name': 'Bicing'}]
----------------------------/n
[{'company': ['PBSC', 'Serco Group plc'], 'href': '/v2/networks/santander-cycles', 'id': 'santander-cycles', 'location': {'city': 'London', 'country': 'GB', 'latitude': 51.51121389999999, 'longitude': -0.1198244}, 'name': 'Santander Cycles'}]
----------------------------/n
[{'company': None, 'ebikes': True, 'gbfs_href': 'https://velib-metropole-opendata.smoove.pro/opendata/Velib_Metropole/gbfs.json', 'href': '/v2/network

In [88]:
get_city_data("Toulouse")

[{'company': ['JCDecaux'],
  'href': '/v2/networks/velo',
  'id': 'velo',
  'license': {'name': 'Open Licence',
   'url': 'https://developer.jcdecaux.com/#/opendata/licence'},
  'location': {'city': 'Toulouse',
   'country': 'FR',
   'latitude': 43.604652,
   'longitude': 1.444209},
  'name': 'Vélô',
  'source': 'https://developer.jcdecaux.com'}]

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [95]:
def get_stations_info(city): 

    station_dict = get_city_data(city)
    if not station_dict: 
        print("Error: no bike company found for {}".format(city))
        return None
    
    network_adress = station_dict[0]['href']
    url = "http://api.citybik.es/{}".format(network_adress)
    return requests.get(url).json()['network']['stations']


In [103]:
city = "Toulouse"
station_info = get_stations_info(city)
station_info

[{'empty_slots': 19,
  'extra': {'address': '6 AV LOUIS BLERIOT',
   'banking': True,
   'bonus': True,
   'last_update': 1670179246000,
   'slots': 20,
   'status': 'OPEN',
   'uid': 197},
  'free_bikes': 1,
  'id': 'f261999c8cf43fd6a8a13a5117f23882',
  'latitude': 43.60038605818475,
  'longitude': 1.464093716152017,
  'name': '00197 - CASERNE PERIGNON',
  'timestamp': '2022-12-04T18:49:45.801000Z'},
 {'empty_slots': 15,
  'extra': {'address': '129 AV DE CASTRES',
   'banking': True,
   'bonus': True,
   'last_update': 1670179212000,
   'slots': 17,
   'status': 'OPEN',
   'uid': 213},
  'free_bikes': 2,
  'id': '4d7eebba14a38524c95d5c8a20e7db1d',
  'latitude': 43.6007551770534,
  'longitude': 1.46751413106582,
  'name': '00213 - PUJOL - CHAUBET',
  'timestamp': '2022-12-04T18:49:45.839000Z'},
 {'empty_slots': 4,
  'extra': {'address': '2 RUE GATIEN ARNOULT',
   'banking': True,
   'bonus': False,
   'last_update': 1670179692000,
   'slots': 15,
   'status': 'OPEN',
   'uid': 55},
  '

In [168]:
def get_available_stations(city = "Toulouse"):
    
    station_info = get_stations_info(city)
    
    station_list = []
    for info in station_info:
        
        a_dict = {
            'Station Name' : info['name'],
            'Address' : info['extra']['address'],
            'empty_slots' : info['empty_slots'],
            'free_bikes' : info['free_bikes'],
            'latitude' : info['latitude'],
            'longitude' : info['longitude'],
            'timestamp' : info['timestamp'],
            'Unique ID' : info['extra']['uid'],
        }
        station_list.append(a_dict)
    
    return pd.DataFrame(station_list)

In [169]:
get_available_stations(city = 'Toulouse').head()

,Station Name,Address,empty_slots,free_bikes,latitude,longitude,timestamp,Unique ID
0,00197 - CASERNE PERIGNON,6 AV LOUIS BLERIOT,19,1,43.600386,1.464094,2022-12-05T00:29:57.070000Z,197
1,00213 - PUJOL - CHAUBET,129 AV DE CASTRES,16,1,43.600755,1.467514,2022-12-05T00:29:57.109000Z,213
2,00055 - SAINT-SERNIN - G. ARNOULT,2 RUE GATIEN ARNOULT,4,11,43.608952,1.441004,2022-12-05T00:29:56.992000Z,55
3,00195 - LARROUSSE - RIEUX,FACE 15 RUE PIERRE LAROUSSE,13,5,43.597235,1.459071,2022-12-05T00:29:56.994000Z,195
4,00280 - CHARBONNIERE,RUE DE LA CHARBONNIERE ENTRE IMP AMPERE ET RUE...,8,10,43.573830,1.448710,2022-12-05T00:29:56.996000Z,280


In [170]:
df = get_available_stations(city = "Toulouse")

In [171]:
df.head()

,Station Name,Address,empty_slots,free_bikes,latitude,longitude,timestamp,Unique ID
0,00197 - CASERNE PERIGNON,6 AV LOUIS BLERIOT,19,1,43.600386,1.464094,2022-12-05T00:29:57.070000Z,197
1,00213 - PUJOL - CHAUBET,129 AV DE CASTRES,16,1,43.600755,1.467514,2022-12-05T00:29:57.109000Z,213
2,00055 - SAINT-SERNIN - G. ARNOULT,2 RUE GATIEN ARNOULT,4,11,43.608952,1.441004,2022-12-05T00:29:56.992000Z,55
3,00195 - LARROUSSE - RIEUX,FACE 15 RUE PIERRE LAROUSSE,13,5,43.597235,1.459071,2022-12-05T00:29:56.994000Z,195
4,00280 - CHARBONNIERE,RUE DE LA CHARBONNIERE ENTRE IMP AMPERE ET RUE...,8,10,43.573830,1.448710,2022-12-05T00:29:56.996000Z,280


In [173]:
df.to_csv('/Users/Oswal/Documents/GitHub/Project-Week-3/notebooks/Toulouse_Data.csv', index = False)

In [143]:
## Method 2: use python city-bikes pkg

import pandas as pd 
import json 
from pandas.io.json import json_normalize
import requests
import citybikes as cb
from datetime import datetime


In [144]:
client = cb.Client()

In [145]:
networks = list(client.networks)

In [146]:
len(networks)

614

In [147]:
len(networks[28].stations)


56

In [148]:
list(networks[0].stations)[0]

{'empty_slots': 9, 'extra': {'address': 'ст. м. Кропоткинская (выход к Гоголевскому бульвару)', 'ebikes': 0, 'electric_free': 0, 'electric_slots': 0, 'normal_bikes': 3, 'normal_free': 9, 'normal_slots': 12, 'slots': 12, 'uid': '0001'}, 'free_bikes': 3, 'id': 'c8a5761ea95e927808b2f37fd65a00f6', 'latitude': 55.7449006, 'longitude': 37.6020183, 'name': 'ст. м. Кропоткинская (выход к Гоголевскому бульвару)', 'timestamp': '2022-10-29T19:26:58.716000Z'}

In [149]:
client = cb.Client()
ToulouseVelo = cb.Network(client, uid='velo')

with open('ToulouseVelo.json', 'w') as f:
    json.dump(ToulouseVelo.data, f, indent=2)

Put your parsed results into a DataFrame.

In [150]:
with open('/Users/Oswal/Documents/GitHub/Project-Week-3/notebooks/ToulouseVelo.json') as Toulouse_data:
    data = json.load(Toulouse_data)

In [161]:
df = pd.json_normalize(data)

In [162]:
df.head()

,company,href,id,name,source,stations,license.name,license.url,location.city,location.country,location.latitude,location.longitude
0,[JCDecaux],/v2/networks/velo,velo,Vélô,https://developer.jcdecaux.com,"[{'empty_slots': 19, 'extra': {'address': '6 A...",Open Licence,https://developer.jcdecaux.com/#/opendata/licence,Toulouse,FR,43.604652,1.444209


In [159]:
df.to_csv

<bound method NDFrame.to_csv of       company               href    id  name                          source  \
0  [JCDecaux]  /v2/networks/velo  velo  Vélô  https://developer.jcdecaux.com   

                                            stations  license.name  \
0  [{'empty_slots': 19, 'extra': {'address': '6 A...  Open Licence   

                                         license.url location.city  \
0  https://developer.jcdecaux.com/#/opendata/licence      Toulouse   

  location.country  location.latitude  location.longitude  
0               FR          43.604652            1.444209  >